In [ ]:
import requests
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from keras.models import Sequential
from keras.layers import LSTM, Dense, BatchNormalization
from keras.losses import mean_squared_error
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam

BASE_URL = 'https://www.alphavantage.co/query?'
API_KEY = 'K37TOUDE2WB29LQJ'
SYMBOL = 'BTCINR'  # Example forex exchange symbol
BATCH_SIZE = 32
NUM_FEATURES = 3
TIME_DELAY = 45  # Time delay in seconds

def get_forex_exchange(symbol):
    params = {
        'function': 'CURRENCY_EXCHANGE_RATE',
        'from_currency': symbol[:3],
        'to_currency': symbol[3:],
        'apikey': API_KEY
    }

    try:
        response = requests.get(BASE_URL, params=params)
        data = response.json()

        forex_exchange = pd.DataFrame({
            'exchange_rate': float(data['Realtime Currency Exchange Rate']['5. Exchange Rate']),
            'bid_price': float(data['Realtime Currency Exchange Rate']['8. Bid Price']),
            'ask_price': float(data['Realtime Currency Exchange Rate']['9. Ask Price'])
        }, index=[pd.to_datetime(data['Realtime Currency Exchange Rate']['6. Last Refreshed'])])
        forex_exchange.index.name = 'time'

        return forex_exchange
    except Exception as e:
        print(f'Error fetching forex exchange details: {e}')
        return pd.DataFrame()

def create_lstm_model(num_features):
    model = Sequential()
    model.add(LSTM(units=400, return_sequences=True, input_shape=(None, num_features), use_bias=True, activation='relu'))
    model.add(BatchNormalization())
    for _ in range(5):
        model.add(LSTM(units=400, return_sequences=True, use_bias=True, activation='relu'))
        model.add(BatchNormalization())
    model.add(LSTM(units=400, return_sequences=False, use_bias=True, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(units=1, activation='linear'))

    optimizer = Adam()
    model.compile(optimizer=optimizer, loss=mean_squared_error, metrics=[RootMeanSquaredError()])

    return model

# Create an empty DataFrame
df = pd.DataFrame(columns=['exchange_rate', 'bid_price', 'ask_price'])

model = create_lstm_model(NUM_FEATURES)

predictions_df = pd.DataFrame(columns=['time', 'real_value', 'predicted_value'])

def train_and_predict(features, labels):
    features = np.expand_dims(features, axis=0)  # Add batch dimension

    model.train_on_batch(features, labels)

    # Make predictions
    predictions = model.predict(features)[0]

    return predictions

def plot_graph():
    plt.figure(figsize=(10, 6))
    plt.plot(predictions_df['time'], predictions_df['real_value'], label='Real Value')
    plt.plot(predictions_df['time'], predictions_df['predicted_value'], label='Predicted Value')
    plt.xlabel('Time')
    plt.ylabel('Exchange Rate')
    plt.title('Real vs Predicted Exchange Rate')
    plt.legend()
    plt.show()

while True:
    forex_exchange = get_forex_exchange(SYMBOL)
    print(forex_exchange)
    if not forex_exchange.empty:
        # Remove duplicates if present
        df = pd.concat([df, forex_exchange]).drop_duplicates()
        num_samples = df.shape[0]
    if num_samples >= BATCH_SIZE:
        start_index = num_samples - BATCH_SIZE
        end_index = num_samples
        features = np.array(df.iloc[start_index:end_index])
        labels = np.array(df['exchange_rate'].iloc[start_index:end_index])

        with ThreadPoolExecutor() as executor:
            predictions = executor.submit(train_and_predict, features, labels).result()

        # Store predictions in DataFrame
        new_predictions = pd.DataFrame({
            'time': df.index[end_index-1],
            'real_value': labels[-1],
            'predicted_value': predictions[-1]
        }, index=[df.index[end_index-1]])

        predictions_df = pd.concat([predictions_df, new_predictions])

        # Plot the graph if DataFrame size is multiple of BATCH_SIZE
        if num_samples % BATCH_SIZE == 0:
            plot_graph()

        time.sleep(TIME_DELAY)
    else:
        time.sleep(30)


In [18]:
import requests
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

BASE_URL = 'https://www.alphavantage.co/query?'
API_KEY = 'Z4XUKBG497A1K6U8'
SYMBOL = 'BTCINR'  # Example forex exchange symbol
BATCH_SIZE = 2
n_features = 3
n_steps = 1
TIME_DELAY = 30  # Time delay in seconds


def get_forex_exchange(symbol):
    params = {
        'function': 'CURRENCY_EXCHANGE_RATE',
        'from_currency': symbol[:3],
        'to_currency': symbol[3:],
        'apikey': API_KEY
    }

    try:
        response = requests.get(BASE_URL, params=params)
        data = response.json()

        forex_exchange = pd.DataFrame({
            'exchange_rate': float(data['Realtime Currency Exchange Rate']['5. Exchange Rate']),
            'bid_price': float(data['Realtime Currency Exchange Rate']['8. Bid Price']),
            'ask_price': float(data['Realtime Currency Exchange Rate']['9. Ask Price'])
        }, index=[pd.to_datetime(data['Realtime Currency Exchange Rate']['6. Last Refreshed'])])
        forex_exchange.index.name = 'time'

        return forex_exchange
    except Exception as e:
        print(f'Error fetching forex exchange details: {e}')
        return pd.DataFrame()


def create_lstm_model(n_steps, n_features):
    model = Sequential()
    model.add(LSTM(units=400, return_sequences=True, input_shape=(
        n_steps, n_features), use_bias=True, activation='relu'))
    model.add(BatchNormalization())
    for _ in range(5):
        model.add(LSTM(units=400, return_sequences=True,
                  use_bias=True, activation='relu'))
        model.add(BatchNormalization())
    model.add(LSTM(units=400, return_sequences=False,
              use_bias=True, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(units=1, activation='linear'))

    optimizer = Adam(0.01)
    model.compile(optimizer=optimizer, loss=mean_squared_error,
                  metrics=[RootMeanSquaredError()])
    model.summary()
    return model


def split_sequence(df1, df2, n_steps, n_features):
    X = []
    y = []
    for i in np.arange(n_steps, len(df2)):
        X.append(df1.iloc[i-n_steps:i,:])
        y.append(df2[i-1])
    X, y = np.array(X), np.array(y)
    X = X.reshape(X.shape[0], n_steps, n_features)
    return X, y


df = pd.DataFrame(columns=['exchange_rate', 'bid_price', 'ask_price'])

model = create_lstm_model(n_steps, n_features)

predictions_df = pd.DataFrame(
    columns=['time', 'real_value', 'predicted_value'])

while True:
    forex_exchange = get_forex_exchange(SYMBOL)
    if not forex_exchange.empty:
        df = pd.concat([df, forex_exchange]).drop_duplicates()
        num_samples = df.shape[0]
    if num_samples > BATCH_SIZE:
        start_index = num_samples - BATCH_SIZE
        end_index = num_samples
        features = df[['exchange_rate', 'bid_price', 'ask_price']].iloc[start_index:end_index]

        labels =  df['exchange_rate'].iloc[start_index+1:end_index+1]

        X, y = split_sequence(features, labels, n_steps, n_features)

        model.train_on_batch(X, y)

        predictions = model.predict(X)

        new_predictions = pd.DataFrame({
            'time': df.index[end_index],
            'real_value': labels[0][-1],
            'predicted_value': predictions[-1]
        }, index=[df.index[end_index-1]])

        predictions_df = pd.concat([predictions_df, new_predictions])

        if num_samples % BATCH_SIZE == 0:
            plt.figure(figsize=(10, 6))
            plt.plot(
                predictions_df['time'], predictions_df['real_value'], label='Real Value')
            plt.plot(
                predictions_df['time'], predictions_df['predicted_value'], label='Predicted Value')
            plt.xlabel('Time')
            plt.ylabel('Exchange Rate')
            plt.title('Real vs Predicted Exchange Rate')
            plt.legend()
            plt.show()

        time.sleep(TIME_DELAY)
    else:
        time.sleep(30)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_28 (LSTM)              (None, 1, 400)            646400    
                                                                 
 batch_normalization_28 (Bat  (None, 1, 400)           1600      
 chNormalization)                                                
                                                                 
 lstm_29 (LSTM)              (None, 1, 400)            1281600   
                                                                 
 batch_normalization_29 (Bat  (None, 1, 400)           1600      
 chNormalization)                                                
                                                                 
 lstm_30 (LSTM)              (None, 1, 400)            1281600   
                                                                 
 batch_normalization_30 (Bat  (None, 1, 400)          

ValueError: ignored

In [16]:
len(df["bid_price"].iloc[0:3])

3